In [1]:
import os
import sys
sys.path.append(os.getcwd())

import polars as pl
import numpy as np
from mars.analysis.profiler import MarsDataProfiler
from mars.utils.logger import set_log_level

# 1. 设置环境
set_log_level("WARNING") # 减少日志干扰，专注看结果

def get_stress_test_data(rows=20000):
    """
    生成包含6种典型'脏数据'场景的测试集，用于验证红绿灯逻辑。
    """
    np.random.seed(42)
    
    # 基础时间轴：前3个月稳定，第4个月(202404)发生漂移
    months = np.random.choice(["202401", "202402", "202403", "202404"], size=rows, p=[0.25, 0.25, 0.25, 0.25])
    
    # --- A. 正常对照组 ---
    # [Case 0] 完美特征: 正态分布，无缺失，无漂移
    f_good = np.random.normal(100, 10, size=rows)

    # --- B. 数据质量 (DQ) 报警测试 ---
    
    # [Case 1] 缺失率爆炸 (Missing Alert)
    # 设定：缺失率 60% (超过一般阈值 50%) -> 预期：Missing Rate 标红
    f_missing = np.random.randn(rows)
    f_missing[np.random.rand(rows) < 0.6] = -999 # 使用 -999 作为自定义缺失
    
    # [Case 2] 唯一值爆炸 (High Cardinality Alert)
    # 设定：ID列，Unique Rate = 100% -> 预期：Unique Rate 标蓝/高亮
    f_id = [f"uid_{i}" for i in range(rows)]
    
    # [Case 3] 单一值报警 (Zero Variance / Top1 Alert)
    # 设定：99.5% 都是 0 -> 预期：Top1 Ratio 标红
    f_const = np.zeros(rows)
    f_const[np.random.rand(rows) < 0.005] = 1 # 只有 0.5% 的非0值

    # --- C. 稳定性 (PSI) 报警测试 ---

    # [Case 4] 数值分布漂移 (PSI Explosion)
    # 设定：前3个月均值0，第4个月均值突变为 5 -> 预期：PSI > 0.25 (红灯)
    f_psi_num = np.random.normal(0, 1, size=rows)
    mask_drift = months == "202404"
    f_psi_num[mask_drift] = np.random.normal(5, 1, size=np.sum(mask_drift))

    # [Case 5] 特殊值占比漂移 (Special Value Shift)
    # 设定：前3个月 -1 占比 1%，第4个月 -1 占比暴涨至 50% -> 预期：PSI 飙升
    # 这是一个非常隐蔽的风险，均值可能变化不大，但结构变了
    f_psi_special = np.random.normal(100, 10, size=rows)
    # 正常月份 1% 是 -1 (异常占位符)
    special_mask_normal = (months != "202404") & (np.random.rand(rows) < 0.01)
    f_psi_special[special_mask_normal] = -1
    # 漂移月份 50% 是 -1
    special_mask_drift = (months == "202404") & (np.random.rand(rows) < 0.50)
    f_psi_special[special_mask_drift] = -1

    return pl.DataFrame({
        "month": months,
        "feature_0_good": f_good,
        "feature_1_high_missing": f_missing,
        "feature_2_high_cardinality": f_id,
        "feature_3_quasi_constant": f_const,
        "feature_4_psi_drift": f_psi_num,
        "feature_5_special_drift": f_psi_special
    })

def run_verification():
    print("🚀 生成测试数据...")
    df = get_stress_test_data()
    
    print("\n🧐 初始化 Profiler (配置特殊值逻辑)...")
    # 关键点：显式区分 Missing (-999) 和 Special (-1)
    profiler = MarsDataProfiler(
        df, 
        custom_missing_values=[-999, "null"],  # 视为缺失
        custom_special_values=[-1]                     # 视为特殊值(参与PSI)
    )

    print("📊 计算画像报告...")
    report = profiler.generate_profile(profile_by="month")
    
    # 获取原始数据进行断言验证
    overview, _, stats_tables = report.get_profile_data()
    psi_table = stats_tables.get('psi')

    print("\n✅ --- 验证结果 (Expectation vs Reality) ---")
    
    # 1. 验证缺失率报警
    missing_rate = overview.filter(pl.col("feature") == "feature_1_high_missing")["missing_rate"][0]
    print(f"[Case 1] High Missing: Expected > 0.5 | Actual: {missing_rate:.2%} -> {'🔴 PASS' if missing_rate > 0.5 else '❌ FAIL'}")

    # 2. 验证常量报警
    top1 = overview.filter(pl.col("feature") == "feature_3_quasi_constant")["top1_ratio"][0]
    print(f"[Case 3] Constant:     Expected > 0.99| Actual: {top1:.2%} -> {'🔴 PASS' if top1 > 0.99 else '❌ FAIL'}")

    if psi_table is not None:
        # 3. 验证数值 PSI 报警
        psi_num = psi_table.filter(pl.col("feature") == "feature_4_psi_drift")["total"][0]
        print(f"[Case 4] Numeric PSI:  Expected > 0.25| Actual: {psi_num:.4f}  -> {'🔴 PASS' if psi_num > 0.25 else '❌ FAIL'}")

        # 4. 验证特殊值 PSI 报警 (这是最难测的)
        psi_special = psi_table.filter(pl.col("feature") == "feature_5_special_drift")["total"][0]
        print(f"[Case 5] Special PSI:  Expected > 0.1 | Actual: {psi_special:.4f}  -> {'🔴 PASS' if psi_special > 0.1 else '❌ FAIL'}")
    
    return report


report = run_verification()


    __________________________________________________________________________
       __  ___ ___    ____  _____
      /  |/  //   |  / __ \/ ___/
     / /|_/ // /| | / /_/ /\__ \ 
    / /  / // ___ |/ _, _/___/ / 
   /_/  /_//_/  |_/_/ |_|/____/  
                                 
    MODELING ANALYSIS RISK SCORE 
    __________________________________________________________________________
    Version: 0.0.1 | Copyright (c) 2026 Christian Li
    High-performance Risk Modeling Toolkit powered by Polars
    __________________________________________________________________________

🚀 生成测试数据...

🧐 初始化 Profiler (配置特殊值逻辑)...
📊 计算画像报告...

✅ --- 验证结果 (Expectation vs Reality) ---
[Case 1] High Missing: Expected > 0.5 | Actual: 59.89% -> 🔴 PASS
[Case 3] Constant:     Expected > 0.99| Actual: 99.50% -> 🔴 PASS
[Case 4] Numeric PSI:  Expected > 0.25| Actual: 2.3844  -> 🔴 PASS
[Case 5] Special PSI:  Expected > 0.1 | Actual: 0.3608  -> 🔴 PASS


In [2]:
report

In [3]:
report.show_overview()

feature,dtype,distribution,missing_rate,zeros_rate,unique_rate,top1_ratio,psi,mean,std,min,max,p25,median,p75,skew,kurtosis
feature_0_good,Float64,▂▂▄▇█▃▂▂,0.00%,0.00%,100.00%,0.01%,nan,99.94,10.02,55.34,144.79,93.03,99.97,106.79,0.01,-0.02
feature_1_high_missing,Float64,▂▂▄▇█▅▂▂,59.89%,0.00%,40.12%,59.89%,nan,0.02,1.00,-3.45,3.28,-0.65,0.02,0.70,-0.03,-0.03
feature_3_quasi_constant,Float64,█______▂,0.00%,99.50%,0.01%,99.50%,nan,0.00,0.07,0.00,1.00,0.00,0.00,0.00,14.11,197.03
feature_4_psi_drift,Float64,▂▄█▄▂▃▃▂,0.00%,0.00%,100.00%,0.01%,nan,1.25,2.38,-3.71,8.43,-0.43,0.43,2.63,0.88,-0.44
feature_5_special_drift,Float64,▂▂▄█▇▄▂▂,0.00%,0.00%,86.93%,13.08%,nan,99.99,10.01,62.99,137.66,93.24,100.01,106.67,-0.02,0.05
feature_2_high_cardinality,String,None,0.00%,0.00%,100.00%,0.01%,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
month,String,None,0.00%,0.00%,0.02%,25.04%,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [4]:
report.show_dq("missing")

feature,dtype,202401,202402,202403,202404,total
feature_0_good,Float64,0.00%,0.00%,0.00%,0.00%,0.00%
feature_1_high_missing,Float64,59.84%,59.38%,61.04%,59.30%,59.89%
feature_3_quasi_constant,Float64,0.00%,0.00%,0.00%,0.00%,0.00%
feature_4_psi_drift,Float64,0.00%,0.00%,0.00%,0.00%,0.00%
feature_5_special_drift,Float64,0.00%,0.00%,0.00%,0.00%,0.00%
feature_2_high_cardinality,String,0.00%,0.00%,0.00%,0.00%,0.00%


In [5]:
report.show_dq("zeros")

feature,dtype,202401,202402,202403,202404,total
feature_0_good,Float64,0.00%,0.00%,0.00%,0.00%,0.00%
feature_1_high_missing,Float64,0.00%,0.00%,0.00%,0.00%,0.00%
feature_3_quasi_constant,Float64,99.62%,99.48%,99.40%,99.52%,99.50%
feature_4_psi_drift,Float64,0.00%,0.00%,0.00%,0.00%,0.00%
feature_5_special_drift,Float64,0.00%,0.00%,0.00%,0.00%,0.00%
feature_2_high_cardinality,String,0.00%,0.00%,0.00%,0.00%,0.00%


In [6]:
report.show_dq("unique")

feature,dtype,202401,202402,202403,202404,total
feature_0_good,Float64,100.00%,100.00%,100.00%,100.00%,100.00%
feature_1_high_missing,Float64,40.18%,40.64%,38.98%,40.72%,40.12%
feature_3_quasi_constant,Float64,0.04%,0.04%,0.04%,0.04%,0.01%
feature_4_psi_drift,Float64,100.00%,100.00%,100.00%,100.00%,100.00%
feature_5_special_drift,Float64,99.22%,98.62%,98.96%,50.83%,86.93%
feature_2_high_cardinality,String,100.00%,100.00%,100.00%,100.00%,100.00%


In [7]:
report.show_dq("top1")

feature,dtype,202401,202402,202403,202404,total
feature_0_good,Float64,0.02%,0.02%,0.02%,0.02%,0.01%
feature_1_high_missing,Float64,59.84%,59.38%,61.04%,59.30%,59.89%
feature_3_quasi_constant,Float64,99.62%,99.48%,99.40%,99.52%,99.50%
feature_4_psi_drift,Float64,0.02%,0.02%,0.02%,0.02%,0.01%
feature_5_special_drift,Float64,0.80%,1.40%,1.06%,49.19%,13.08%
feature_2_high_cardinality,String,0.02%,0.02%,0.02%,0.02%,0.01%


In [8]:
report.show_trend("psi")

feature,dtype,202401,202402,202403,202404,total,group_mean,group_var,group_cv
feature_0_good,Float64,0.00,0.01,0.01,0.01,0.00,0.01,0.0000,0.7094
feature_1_high_missing,Float64,0.00,0.00,0.01,0.00,0.00,0.00,0.0000,0.6841
feature_2_high_cardinality,String,0.00,10.54,10.54,10.55,3.91,7.91,27.7886,0.6667
feature_3_quasi_constant,Float64,0.00,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000
feature_4_psi_drift,Float64,0.00,0.00,0.00,20.79,2.38,5.20,108.0789,1.9995
feature_5_special_drift,Float64,0.00,0.01,0.00,2.32,0.36,0.58,1.3406,1.9841


In [9]:
report.show_trend("mean")

feature,dtype,202401,202402,202403,202404,total,group_mean,group_var,group_cv
feature_0_good,Float64,100.19,99.73,99.94,99.89,99.94,99.94,0.0364,0.0019
feature_1_high_missing,Float64,0.06,-0.00,0.02,0.01,0.02,0.02,0.0009,1.3307
feature_3_quasi_constant,Float64,0.00,0.01,0.01,0.00,0.00,0.00,0.0000,0.1845
feature_4_psi_drift,Float64,-0.01,-0.00,0.00,5.01,1.25,1.25,6.2714,2.0028
feature_5_special_drift,Float64,99.91,100.15,100.00,99.80,99.99,99.96,0.0224,0.0015
feature_2_high_cardinality,String,nan,nan,nan,nan,nan,0.00,0.0000,0.0000


In [10]:
report.show_trend("max")

feature,dtype,202401,202402,202403,202404,total,group_mean,group_var,group_cv
feature_0_good,Float64,144.79,136.11,136.92,139.42,144.79,139.31,15.3334,0.0281
feature_1_high_missing,Float64,3.18,3.28,2.99,2.82,3.28,3.07,0.0426,0.0673
feature_3_quasi_constant,Float64,1.00,1.00,1.00,1.00,1.00,1.00,0.0000,0.0000
feature_4_psi_drift,Float64,3.71,3.77,4.20,8.43,8.43,5.03,5.1888,0.4531
feature_5_special_drift,Float64,133.89,132.86,137.66,134.44,137.66,134.71,4.2924,0.0154
feature_2_high_cardinality,String,nan,nan,nan,nan,nan,0.00,0.0000,0.0000


In [11]:
report.show_trend("min")

feature,dtype,202401,202402,202403,202404,total,group_mean,group_var,group_cv
feature_0_good,Float64,65.05,64.67,55.34,61.44,55.34,61.63,20.1659,0.0729
feature_1_high_missing,Float64,-3.45,-3.28,-3.43,-3.29,-3.45,-3.36,0.0080,0.0267
feature_3_quasi_constant,Float64,0.00,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000
feature_4_psi_drift,Float64,-3.63,-3.71,-3.40,1.22,-3.71,-2.38,5.7625,1.0094
feature_5_special_drift,Float64,66.15,65.66,62.99,64.64,62.99,64.86,1.9565,0.0216
feature_2_high_cardinality,String,nan,nan,nan,nan,nan,0.00,0.0000,0.0000


In [12]:
report.write_excel("mars_demo_report.xlsx")

[MARS] 2026-01-16 01:30:33 - ERROR - ❌ Failed to export Excel: No module named 'xlsxwriter'


In [13]:
overview, dq_tables, stat_tables = report.get_profile_data()
overview

feature,dtype,distribution,missing_rate,zeros_rate,unique_rate,top1_ratio,psi,mean,std,min,max,p25,median,p75,skew,kurtosis
str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""feature_0_good""","""Float64""","""▂▂▄▇█▃▂▂""",0.0,0.0,1.0,0.00005,null,99.938734,10.019129,55.343961,144.790843,93.029211,99.970241,106.790119,0.006088,-0.017714
"""feature_1_high_missing""","""Float64""","""▂▂▄▇█▅▂▂""",0.5989,0.0,0.40115,0.5989,null,0.022562,1.003979,-3.448043,3.2803,-0.653159,0.018441,0.698605,-0.032651,-0.029669
"""feature_3_quasi_constant""","""Float64""","""█______▂""",0.0,0.99505,0.0001,0.99505,null,0.00495,0.070184,0.0,1.0,0.0,0.0,0.0,14.107628,197.025177
"""feature_4_psi_drift""","""Float64""","""▂▄█▄▂▃▃▂""",0.0,0.0,1.0,0.00005,null,1.246642,2.382778,-3.70515,8.428319,-0.432222,0.426586,2.634416,0.878427,-0.437719
"""feature_5_special_drift""","""Float64""","""▂▂▄█▇▄▂▂""",0.0,0.0,0.8693,0.13075,null,99.988006,10.008061,62.988947,137.662336,93.239218,100.006717,106.667331,-0.016807,0.046669
"""feature_2_high_cardinality""","""String""",null,0.0,0.0,1.0,0.00005,null,null,null,null,null,null,null,null,null,null
"""month""","""String""",null,0.0,0.0,0.0002,0.25035,null,null,null,null,null,null,null,null,null,null


In [14]:
# 伪代码逻辑
def filter_bad_features(df):
    
    # 1. 删掉“太无聊”的列 (准常量)
    # 只要 Top1 占比太高，不管它是啥类型，直接删
    top1_ratio = 0.96
    if top1_ratio > 0.95: 
        return "DROP (Quasi-Constant)"

    # 2. 删掉“太杂乱”的 ID 列 (高基数)
    # 只有当它是字符串类型，且几乎都不重复时，才删
    # (数值型的高Unique是好事，不能删！)
    unique_rate = 1.0
    is_string_type = True
    if unique_rate > 0.95 and is_string_type:
        return "DROP (ID Column)"
        
    return "KEEP"

In [ ]:
import polars as pl
import pandas as pd
import numpy as np
import time
import psutil
import os
import gc
import sys
from typing import Tuple, Union

from mars.analysis.profiler import MarsDataProfiler
from mars.utils.logger import set_log_level, logger

# ==========================================
# ⚙️ Stress Test Configuration
# ==========================================
set_log_level("WARNING")  # Reduce IO interference

# Standard medium-sized risk dataset configuration (200k rows x 2000 cols)
N_ROWS: int = 200000
N_COLS: int = 5000
N_CATS: int = 50
N_GROUPS: int = 12

class Colors:
    GREEN = '\033[92m'
    RED = '\033[91m'
    CYAN = '\033[96m'
    BOLD = '\033[1m'
    RESET = '\033[0m'

def get_memory_usage() -> float:
    """Get current process memory (MB)"""
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024

def generate_data() -> pl.DataFrame:
    """Fast large-scale test data generation"""
    print(f"{Colors.CYAN}🚀 [DataGen] Generating {N_ROWS:,} rows x {N_COLS} cols...{Colors.RESET}")
    start = time.time()
    
    # 1. Numerical columns (Matrix generation is faster)
    n_num = N_COLS - N_CATS
    # Using float32 to save memory for the demo, can be float64
    data = (np.random.randn(N_ROWS, n_num).astype(np.float32) * 10) + 100
    
    # Inject missing values (-999)
    mask = np.random.random(data.shape) < 0.1
    data[mask] = -999
    
    data_dict = {f"num_{i}": data[:, i] for i in range(n_num)}
    
    # 2. Categorical columns
    cats = ["A", "B", "C", "D", "E", "unknown", None]
    for i in range(N_CATS):
        data_dict[f"cat_{i}"] = np.random.choice(cats, size=N_ROWS).tolist()
        
    # 3. Group column
    groups = [f"2023{m:02d}" for m in range(1, N_GROUPS + 1)]
    data_dict["month"] = np.random.choice(groups, size=N_ROWS).tolist()
    
    df = pl.DataFrame(data_dict)
    size_mb = df.estimated_size('mb')
    print(f"✅ Data Ready! Size: {size_mb:.2f} MB | Time: {time.time()-start:.2f}s")
    
    # Check if memory is sufficient for conversion later
    if size_mb * 3 > psutil.virtual_memory().available / 1024 / 1024:
        print(f"{Colors.RED}⚠️ Warning: Dataset might be too large for Pandas conversion on this machine.{Colors.RESET}")
        
    return df

def run_benchmark_round(df: Union[pl.DataFrame, pd.DataFrame], backend: str) -> Tuple[float, float, float]:
    """Execute a single round of stress testing"""
    print(f"\n🔹 Testing Backend: {Colors.BOLD}{backend}{Colors.RESET}")
    print("-" * 60)
    
    # Configuration overrides for benchmark (disable sparklines for pure calc speed)
    # If you want to test sparkline performance, remove "enable_sparkline": False
    bench_config = {
        "enable_sparkline": False,
        "stat_metrics": ["psi", "mean", "std", "min", "max", "p25", "median", "p75", "skew", "kurtosis"]
        } 

    # 1. Initialization
    gc.collect()
    t0 = time.time()
    # Initialize Engine
    profiler = MarsDataProfiler(df, custom_missing_values=[-999, "unknown"])
    t_init = time.time() - t0
    print(f"   1. Init Engine       : {t_init:.4f} s")
    
    # 2. Overview Only (Simulates old get_report)
    # Using profile_by=None to get global stats
    t1 = time.time()
    _ = profiler.generate_profile(profile_by=None, config_overrides=bench_config)
    t_report = time.time() - t1
    print(f"   2. Full Overview     : {t_report:.4f} s")
    
    # 3. Group Profile (Stability Analysis)
    # Using profile_by="month"
    t2 = time.time()
    _ = profiler.generate_profile(
        profile_by="month", 
        config_overrides=bench_config # Reuse config to keep sparklines off
    )
    t_profile = time.time() - t2
    print(f"   3. Group Profile (by): {t_profile:.4f} s")
    
    return t_init, t_report, t_profile

def print_final_report(pl_times, pd_times):
    """Print comparison report"""
    stages = ["Initialization", "Get Full Overview", "Generate Group Profile"]
    
    print(f"\n{Colors.BOLD}{'🏆 BENCHMARK RESULTS (Time in Seconds)':^65}{Colors.RESET}")
    print("=" * 65)
    print(f"| {'Stage':<24} | {'Polars':<10} | {'Pandas':<10} | {'Speedup':<10} |")
    print("|" + "-"*26 + "+" + "-"*12 + "+" + "-"*12 + "+" + "-"*12 + "|")
    
    for stage, t_pl, t_pd in zip(stages, pl_times, pd_times):
        # Calculate speedup
        speedup = t_pd / t_pl if t_pl > 0 else 0
        
        # Color code the winner
        if t_pl < t_pd:
            pl_str = f"{Colors.GREEN}{t_pl:.4f}{Colors.RESET}"
            pd_str = f"{t_pd:.4f}"
        else:
            pl_str = f"{t_pl:.4f}"
            pd_str = f"{Colors.GREEN}{t_pd:.4f}{Colors.RESET}"
            
        print(f"| {stage:<24} | {pl_str:<19} | {pd_str:<19} | {speedup:>9.1f}x |")
    print("=" * 65)
    print(f"{Colors.CYAN}* Speedup > 1.0x means Polars is faster.{Colors.RESET}")
    print(f"{Colors.CYAN}* Note: Sparklines were disabled for pure calculation benchmark.{Colors.RESET}\n")

# 1. Generate Data (Polars Native)
df_pl = generate_data()

# 2. Run Polars Benchmark
pl_results = run_benchmark_round(df_pl, "Polars (Native)")

# 3. Convert to Pandas for Comparison
print(f"\n{Colors.CYAN}🔄 Converting to Pandas for compatibility test...{Colors.RESET}")
try:
    t_conv = time.time()
    df_pd = df_pl.to_pandas()
    print(f"   Conversion time: {time.time() - t_conv:.2f}s")
    
    # Explicitly delete Polars DF to free memory if tight
    # del df_pl 
    gc.collect()
except MemoryError:
    print(f"{Colors.RED}❌ OOM during conversion! Skipping Pandas test.{Colors.RESET}")
    sys.exit(1)
    
# 4. Run Pandas Benchmark
pd_results = run_benchmark_round(df_pd, "Pandas (Compat)")

# 5. Show Summary
print_final_report(pl_results, pd_results)


    __________________________________________________________________________
       __  ___ ___    ____  _____
      /  |/  //   |  / __ \/ ___/
     / /|_/ // /| | / /_/ /\__ \ 
    / /  / // ___ |/ _, _/___/ / 
   /_/  /_//_/  |_/_/ |_|/____/  
                                 
    MODELING ANALYSIS RISK SCORE 
    __________________________________________________________________________
    Version: 0.0.1 | Copyright (c) 2026 Christian Li
    High-performance Risk Modeling Toolkit powered by Polars
    __________________________________________________________________________

🚀 [DataGen] Generating 200,000 rows x 5000 cols...
✅ Data Ready! Size: 3794.05 MB | Time: 36.75s

🔹 Testing Backend: Polars (Native)
------------------------------------------------------------
   1. Init Engine       : 0.0010 s
   2. Full Overview     : 35.4596 s
